In [1]:
import numpy as np
from PIL import Image
import cv2
import csv
import os
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
from tracktor.utils import class_index

In [2]:
folder = "../../data/strandvejen_faxe"
file_names = sorted(os.listdir(folder), key = lambda x: x[:4])
file_path = [folder + "/" + name for name in file_names]

In [3]:
model_output = pd.read_csv(
    "../../output/custom/results/strandvejen_faxe.csv",
    header=None)
model_output.columns = ["frame", "id", "bb_left", "bb_top", "bb_width", "bb_height", "confidence", "x", "y", "z", "label_index"]
model_output['label'] = (model_output['label_index']).map(lambda x: class_index[str(x)])

In [4]:
def get_img(img_path):
    img = cv2.imread(img_path)
    #img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    #img = cv2.resize(img, (512, 512))
    return img

def hist_eq(img):
    hist,bins = np.histogram(img.flatten(),256,[0,256])
    cdf = hist.cumsum()
    cdf_normalized = cdf * hist.max()/ cdf.max()
    cdf_m = np.ma.masked_equal(cdf,0)
    cdf_m = (cdf_m - cdf_m.min())*255/(cdf_m.max()-cdf_m.min())
    cdf = np.ma.filled(cdf_m,0).astype('uint8')
    return cdf[img]

def print_img(img):
    plt.figure(figsize=(15, 15))
    plt.imshow(img, cmap='Greys_r')
    
def draw_bounding(img, x1, x2, y1, y2, label):
    # draw a rectangle on the image
    h, w = img.shape[:2]
    cv2.putText(img, label , (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,12), 1)
    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 255, 255), 2)

In [5]:
model_output.head(3)

,frame,id,bb_left,bb_top,bb_width,bb_height,confidence,x,y,z,label_index,label
0,1,1,429.018219,883.143433,274.919708,45.375000,0.748359,-1,-1,-1,10,crack
1,2,2,423.240234,901.539917,286.609619,52.563293,0.837370,-1,-1,-1,10,crack
2,2,3,980.716675,932.912292,44.236145,29.497864,0.484727,-1,-1,-1,2,crack


In [6]:
i = 10
one_frame = get_img(file_path[i])
frame_boxes_df = model_output[model_output["frame"] == i]
frame_boxes_df

,frame,id,bb_left,bb_top,bb_width,bb_height,confidence,x,y,z,label_index,label
11,10,4,535.955811,663.10199,462.182739,362.89801,0.153277,-1,-1,-1,10,crack


In [7]:
output_img = one_frame.copy()
for i, row in frame_boxes_df.iterrows():
    x1 = float(row.bb_left) - 1
    y1 = float(row.bb_top) - 1
    x2 = float(row.bb_width) - 1 + x1
    y2 = float(row.bb_height) - 1 + y1
    draw_bounding(output_img, int(x1), int(x2), int(y1), int(y2), row.label)
print_img(output_img)

In [8]:
fourcc = cv2.VideoWriter_fourcc(*"MJPG")
writer = cv2.VideoWriter("tracktor_video.avi", fourcc, 30, (1024, 1024), True)
for i in range(len(model_output['frame'].unique())):
    one_frame = get_img(file_path[i])
    frame_boxes_df = model_output[model_output["frame"] == i]
    output_img = one_frame.copy()
    for _, row in frame_boxes_df.iterrows():
        x1 = float(row.bb_left) - 1
        y1 = float(row.bb_top) - 1
        x2 = float(row.bb_width) - 1 + x1
        y2 = float(row.bb_height) - 1 + y1
        draw_bounding(
            output_img,
            int(x1), int(x2), int(y1), int(y2),
            f"[{row.id}] {row.label} - {round(row.confidence*100)}%]"
        )
    writer.write(output_img)
writer.release()


In [9]:
model_output["label"].unique()

array(['crack', 'area patch', 'hole'], dtype=object)

In [10]:
frame_boxes_df

,frame,id,bb_left,bb_top,bb_width,bb_height,confidence,x,y,z,label_index,label
1010,335,366,510.21756,648.123901,478.708405,375.89386,0.695087,-1,-1,-1,10,crack
